### Import packages

In [14]:
import os
import ee
import geemap
import matplotlib.pyplot as plt
from geemap import cartoee
from zipfile import ZipFile
from urllib.request import urlretrieve
# from urllib.parse import urljoin
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon
from pprint import pprint
from datetime import datetime as dt

In [15]:
ee.Initialize()

### Save paths and urls to variables

In [16]:
# inputdir = "../data/"
# ne_countries_zip = "ne_50m_admin_0_countries.zip"
# # ne_provinces_zip = "ne_10m_admin_1_states_provinces_scale_rank.zip"

# # ne_10m_cultural_download_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/"
# ne_50m_cultural_download_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/"

# ne_countries_shp = "ne_50m_admin_0_countries.shp"
# # ne_provinces_shp = "ne_10m_admin_1_states_provinces_scale_rank.shp"

# # define urls and paths
# countries_zip_path = os.path.join(inputdir, ne_countries_zip)
# # provinces_zip_path = os.path.join(inputdir, ne_provinces_zip)

# ne_countries_zip_url = ne_50m_cultural_download_url + ne_countries_zip # explore why urljoin removes one /
# # ne_provinces_zip_url = ne_50m_cultural_download_url + ne_provinces_zip

# countries_shp_path = os.path.join(inputdir, ne_countries_shp)
# # provinces_shp_path = os.path.join(inputdir, ne_provinces_shp)

### Download country borders for zonal statistics - source Natural Earth 

In [17]:
# def get_zip(target_dir, path, url):
#     """
#     Check if file in `path` exists, otherwise download it from `url`
#     """
#     try:
#         with ZipFile(path, 'r') as zip:
#             print('Extracting all files...')
#             zip.extractall(target_dir)
#             print('Extracted!')
#     except FileNotFoundError:
#         print('Zip file not found, downloading now...')
#         urlretrieve(url, path)
#         print('Downloaded!')
#         with ZipFile(path, 'r') as zip:
#             # extracting all files
#             print('Extracting all files...')
#             zip.extractall(target_dir)
#             print('Extracted!')
#             # print content of target folder
#         print(os.listdir(target_dir))

In [18]:
# get_zip(inputdir, countries_zip_path, ne_countries_zip_url)
# get_zip(inputdir, provinces_zip_path, ne_provinces_zip_url)

Extracting all files...
Extracted!


### Subset world countries to include only Europe (excl. Russia)

In [19]:
# countries = gpd.read_file(countries_shp_path)
# # countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
# # prov = gpd.read_file(provinces_shp_path)
# countries.columns = [x.lower() for x in countries.columns]
# # prov.columns = [x.lower() for x in prov.columns]

# # subset eu countries 
# eu = countries.loc[(countries.continent == "Europe") & (countries.name != "Russia")]
# # NOTE: Bug in countries df - French Guyana is France, Svalbard Islands are Norway, etc

# # Define custom poly for EU
# eu_poly = Polygon([
#     (-18.225, 46.429),
#     (-8.325, 65,554),
#     (28.575, 74.329),
#     (34.650, 70.279),
#     (41.625, 45.754),
#     (26.550, 32.254),
#     (-10.800, 34.279),
#     (-18.225, 46.429)
# ])

# # Explode geometries and mask the EU
# eu_explo = eu.explode()
# eu = eu_explo[eu_explo.intersects(eu_poly)].dissolve(by='name')
# eu_fc = geemap.geopandas_to_ee(eu)

### Clip provinces with EU countries # too many - look for shape of regions

In [20]:
# eu_prov = gpd.overlay(prov, eu, how="intersection")
# eu_prov_fc = geemap.geopandas_to_ee(eu_prov)

In [21]:
# ax = eu.plot(figsize=(25, 10), color='gray')
# eu_prov.loc[eu_prov.name_en == "Italy"].plot(ax=ax, color="red")

### Import country borders frem GEE collection

In [84]:
world = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# pprint(world.limit(1).getInfo()['features'][0]['properties'])
eu = world.filter(ee.Filter.eq('wld_rgn', 'Europe'))
eu = eu.filter(ee.Filter.eq('country_na', 'Russia').Not())
eu = eu.filter(ee.Filter.eq('country_na', 'Svalbard').Not())
cyp = world.filter(ee.Filter.eq('country_na', 'Cyprus'))
eu = ee.FeatureCollection(eu.merge(cyp))
eu_size = eu.size()
eu_size.getInfo()

53

### Initialize leaflet Map centered on EU

In [85]:
# initialize map
center_map = [48, 8]
zoom_default = 4
map0 = geemap.Map(center=center_map, zoom=zoom_default)
# add basemap
map0.add_basemap("CartoDB.DarkMatter")
# add country borders
map0.addLayer(eu_cyp, {}, "World boundaries")
map0

Map(center=[48, 8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Import relavant satellite data collections

In [23]:
start_date = '2015-04-01'
end_date = '2021-04-22'

# subsurface soil moisture # 10km res - 2 or 3 days rev
susm = ee.ImageCollection("NASA_USDA/HSL/SMAP10KM_soil_moisture")\
    .select("susm")\
    .filterDate(start_date, end_date)

susm_vps = {
    "min": 0.0,
    "max": 274.6,
    "palette": ['ff0303', 'efff07', 'efff07', '418504', '0300ff']
}

# # import a land cover # all bands at 500 m res
# lcc = ee.ImageCollection('MODIS/006/MCD12Q1')\
#     .select("LC_Type1")\
#     .filterDate(start_date, end_date)

# lcc_vps = {
#     "min": 1.0,
#     "max": 17.0,
#     "palette": [
#         '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
#         'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
#         '69fff8', 'f9ffa4', '1c0dff'
#     ]
# }

# # import a land surface temperature # 1km res - 1 day rev
# lst = ee.ImageCollection('MODIS/006/MOD11A1')\
#     .select('LST_Day_1km')\
#     .filterDate(start_date, end_date)

# lst_vps = {
#     "min": 13000.0,
#     "max": 16500.0,
#     "palette": [
#         '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
#         '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
#         '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
#         'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
#         'ff0000', 'de0101', 'c21301', 'a71001', '911003'
#     ]
# }


# # import a ground elevation - 30m res
# srtm = ee.Image('USGS/SRTMGL1_003')

### Explore SMAP collection

In [54]:
# Get number of images
size = susm.size()

# Gate date range
rango = susm.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])

init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d %H:%M:%S')

last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.
last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d %H:%M:%S')

# Get proprierty names
properties = susm.first().propertyNames()
properties.getInfo()

print('N images in SMAP:', size.getInfo(), '\nDate range SMAP: ', init_date_f, ' - ', last_date_f) 

N images in SMAP: 738 
Date range SMAP:  2015-04-02 12:00:00  -  2021-04-21 12:00:00


### Add first image of SMAP collection, land cover, etc

In [24]:
susm1 = ee.Image(susm.first())
# lst1 = ee.Image(lst.first())
# lcc1 = ee.Image(lcc.first())
# Map.addLayer(susm1, susm_vps, 'First image SMAP')
# Map.addLayer(lst1, lst_vps, 'First image LST')
# Map.addLayer(lcc1, lcc_vps, 'First image LCC')
# Map

### Define region of interest for time series analysis

In [25]:
# xmin, ymin, xmax, ymax 
eu_bbox = [-18, 32, 41, 74]

# Check grid size for spatial re-sampling
ax = cartoee.get_map(susm1, region=eu_bbox, vis_params=susm_vps)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[2, 2], linestyle=":")
# add colorbar
cartoee.add_colorbar(ax, susm_vps, loc='right')
# set title
ax.set_title(label='Sub-Surface Soil Moisture over EU', fontsize=15)
plt.show()

NameError: name 'susm1' is not defined

### Extract data for time series analysis

In [ ]:
# test function on single image
img = susm.first()
stats_img = img.reduceRegions(
    reducer=ee.Reducer.mean(), 
    collection=eu_fc, 
    scale=10000)
# pprint(stats_img.limit(1).getInfo()['features'][0]['properties'])

In [61]:
# map function over ImageCollection
def regMean(image):
    return image.addBands(
        image.reduceRegions(
            reducer=ee.Reducer.mean(),
            collection=eu_fc,
            scale=10000))

susm_mean = susm.map(regMean)
susm_mean.name() # the method .name() returns the type of EE object

'ImageCollection'

In [ ]:
images = susm_mean.map(lambda feature: ee.Image(ee.Number(susm_mean.get('mean'))))